# 📊 Telecom X – Parte 2: Predicción de Cancelación (Churn)

Notebook desarrollado como parte del Challenge de **Telecom X**.

Incluye: carga de datos, preprocesamiento, EDA, modelado, evaluación y conclusiones estratégicas.

In [ ]:

# Importar librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


## 1. Carga de datos

Se carga el archivo CSV previamente tratado en la Parte 1 (datos limpios y organizados).

In [ ]:

# ⚠️ Cambiar por el nombre correcto del archivo limpio exportado en la Parte 1
df = pd.read_csv("TelecomX_clean.csv")

df.head()


## 2. Limpieza de columnas irrelevantes

Eliminamos identificadores únicos como `customerID`, que no aportan valor predictivo.

In [ ]:

# Eliminamos columnas irrelevantes
df = df.drop(columns=["customerID"], errors="ignore")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()


## 3. Transformación de variables categóricas

Se aplica One-Hot Encoding a variables categóricas. La variable target `Churn` se convierte a binaria (Yes=1, No=0).

In [ ]:

# Variable target
y = df["Churn"].map({"Yes": 1, "No": 0})
X = df.drop(columns=["Churn"])

# One-hot encoding
X = pd.get_dummies(X, drop_first=True)
X.head()


## 4. Balance de clases

Se revisa la proporción de clientes que cancelaron vs. los que permanecieron.

In [ ]:

churn_dist = y.value_counts(normalize=True) * 100
print("Distribución de Churn (%):\n", churn_dist)

churn_dist.plot(kind="bar", color=["skyblue","salmon"])
plt.title("Proporción de clientes: Churn vs No Churn")
plt.ylabel("%")
plt.xticks(rotation=0)
plt.show()


## 5. Correlación entre variables numéricas

Se visualizan las correlaciones para identificar variables relevantes.

In [ ]:

df_corr = df[["tenure", "Charges.Monthly", "Charges.Total", "Churn"]].copy()
df_corr["Churn"] = df_corr["Churn"].map({"Yes": 1, "No": 0})

plt.figure(figsize=(8,6))
sns.heatmap(df_corr.corr(), annot=True, cmap="coolwarm", center=0)
plt.title("Matriz de correlación")
plt.show()


## 6. Exploración de variables clave vs Churn

Se analizan `tenure` y `Charges.Total` respecto a la cancelación.

In [ ]:

# Boxplot tenure
sns.boxplot(x=y, y=df["tenure"], palette="Set2")
plt.title("Tiempo de contrato vs Churn")
plt.show()

# Boxplot gasto total
sns.boxplot(x=y, y=df["Charges.Total"], palette="Set1")
plt.title("Gasto total vs Churn")
plt.show()

# Scatter tenure vs gasto total
sns.scatterplot(x=df["tenure"], y=df["Charges.Total"], hue=y, alpha=0.6)
plt.title("Relación Tenure y Gasto Total por Cancelación")
plt.show()


## 7. Normalización de variables numéricas

Se normalizan variables numéricas solo para modelos sensibles a escala (ej. Regresión Logística).

In [ ]:

num_cols = ["tenure", "Charges.Monthly", "Charges.Total"]
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[num_cols] = scaler.fit_transform(X_scaled[num_cols])


## 8. División en train y test

Se separan los datos en 70% entrenamiento y 30% prueba, manteniendo la proporción de clases (stratify).

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train_scaled, X_test_scaled, _, _ = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

print("Train:", X_train.shape, "Test:", X_test.shape)


## 9. Modelado

- **Modelo 1:** Regresión Logística (con datos normalizados).
- **Modelo 2:** Random Forest (sin normalización).

In [ ]:

log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_scaled, y_train)

rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)
rf_clf.fit(X_train, y_train)


## 10. Evaluación de modelos

Se calculan métricas: Accuracy, Precisión, Recall, F1-score y Matriz de confusión.

In [ ]:

def evaluar_modelo(modelo, X_tr, X_te, y_tr, y_te, nombre):
    print(f"===== {nombre} =====")
    y_pred_train = modelo.predict(X_tr)
    y_pred_test = modelo.predict(X_te)
    
    acc = accuracy_score(y_te, y_pred_test)
    prec = precision_score(y_te, y_pred_test)
    rec = recall_score(y_te, y_pred_test)
    f1 = f1_score(y_te, y_pred_test)
    
    print(f"Accuracy : {acc:.4f}")
    print(f"Precisión: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1-score : {f1:.4f}\n")
    
    print("Reporte de clasificación:")
    print(classification_report(y_te, y_pred_test))
    
    cm = confusion_matrix(y_te, y_pred_test)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Churn","Churn"], yticklabels=["No Churn","Churn"])
    plt.title(f"Matriz de confusión - {nombre}")
    plt.show()
    
    print("="*40)

# Evaluamos ambos
evaluar_modelo(log_reg, X_train_scaled, X_test_scaled, y_train, y_test, "Regresión Logística")
evaluar_modelo(rf_clf, X_train, X_test, y_train, y_test, "Random Forest")


## 11. Importancia de variables

- **Regresión Logística:** análisis de coeficientes.
- **Random Forest:** importancia de variables según impureza.

In [ ]:

# Coeficientes RL
coef = pd.DataFrame({
    "Variable": X_train_scaled.columns,
    "Coeficiente": log_reg.coef_[0]
}).sort_values(by="Coeficiente", ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(y="Variable", x="Coeficiente", data=coef.head(10), palette="coolwarm")
plt.title("Top variables - Regresión Logística")
plt.show()

# Importancia RF
importancias = pd.DataFrame({
    "Variable": X_train.columns,
    "Importancia": rf_clf.feature_importances_
}).sort_values(by="Importancia", ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(y="Variable", x="Importancia", data=importancias.head(10), palette="viridis")
plt.title("Top variables - Random Forest")
plt.show()


## 12. Conclusiones Estratégicas

📌 **Factores más influyentes en la cancelación:**
- Contrato mes a mes (riesgo alto).
- Clientes con bajo tenure (poca antigüedad).
- Método de pago Electronic Check.
- Cargos mensuales altos.
- Ausencia de servicios de valor agregado.

📌 **Estrategias de retención propuestas:**
- Migrar clientes de mes a mes a contratos de 1–2 años con incentivos.
- Contacto proactivo en primeros 90 días.
- Ofrecer bundles/planes optimizados para clientes con MonthlyCharges altos.
- Incentivar métodos de pago estables (tarjeta/transferencia).
- Promover servicios de valor agregado como seguridad online y soporte técnico.

👉 Con estas acciones, se espera reducir el churn y aumentar la fidelidad del cliente.